# glm 4v 9b

## 1 模型加载

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

import torch
from threading import Thread
from transformers import (
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    TextIteratorStreamer, AutoModel, BitsAndBytesConfig
)

from PIL import Image

MODEL_PATH = "/opt/Data/ModelWeight/THUDM/glm-4v-9b"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    encode_special_tokens=True
)

## For INT4 inference
model = AutoModel.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    device_map="auto", # 多卡部署
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True
).eval()

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

## 2 模型调用

调用函数声明

In [17]:
def buildPrompt(user_input:str, history, image = None):
    history.append([user_input, ""])
    messages = []
    for idx, (user_msg, model_msg) in enumerate(history):
        if idx == len(history) - 1 and not model_msg:
            messages.append({"role": "user", "content": user_msg})
            if image:
                messages[-1].update({"image": image})
                uploaded = True
            break
    
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
    
        if model_msg:
            messages.append({"role": "assistant", "content": model_msg})

    return messages

In [18]:
def runChat(messages):    
    model_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True
    ).to(next(model.parameters()).device)
    
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer,
        timeout=60,
        skip_prompt=True,
        skip_special_tokens=True
    )
    
    generate_kwargs = {
        **model_inputs,
        "streamer": streamer,
        "max_new_tokens": 4096,
        "do_sample": True,
        "top_p": 0.8,
        "temperature": 0.6,
        "stopping_criteria": StoppingCriteriaList([StopOnTokens()]),
        "repetition_penalty": 1.2,
        "eos_token_id": [151329, 151336, 151338],
    }
    
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    print("GLM-4V:", end="", flush=True)
    
    for new_token in streamer:
        if new_token:
            print(new_token, end="", flush=True)
            history[-1][1] += new_token
    
    history[-1][1] = history[-1][1].strip()

构建prompt 生成文本

In [20]:
history = []
image = None
image_path = "/opt/WorkSpace/GLM-4/image/Doraemon.jpg"
try:
    image = Image.open(image_path).convert("RGB")
except:
    print("Invalid image path. Continuing with text conversation.")


user_input = "这是什么？"

ms = buildPrompt(user_input, history, image)
runChat(ms)

GLM-4V:这是哆啦A梦，它是日本漫画《哆啦A梦》的主角。这只机器猫来自22世纪的未来世界，拥有许多神奇的道具帮助主人野比大雄解决各种问题。《哆啦A梦》自1969年首播以来深受全球观众喜爱。
哆啦A梦以其圆圆的脸庞、蓝色的身体和红色的鼻子著称于世；它穿着一件黄色的铃铛挂在脖子上和一个红色领带（在一些版本中是蓝色）。这个角色以它的乐观态度和对朋友的忠诚而广受好评。

除了作为一部著名的动画系列外，《哆啦A梦》还衍生出了多部电影、电视剧集以及一系列的玩具产品和其他媒体形式的产品开发项目。“哆啦A梦”已经成为一个国际性的文化现象和品牌标识的一部分。

In [22]:
user_input = "还有呢？"
print(history)
ms = buildPrompt(user_input, history)
runChat(ms)

[['这是什么？', '这是哆啦A梦，它是日本漫画《哆啦A梦》的主角。这只机器猫来自22世纪的未来世界，拥有许多神奇的道具帮助主人野比大雄解决各种问题。《哆啦A梦》自1969年首播以来深受全球观众喜爱。\n哆啦A梦以其圆圆的脸庞、蓝色的身体和红色的鼻子著称于世；它穿着一件黄色的铃铛挂在脖子上和一个红色领带（在一些版本中是蓝色）。这个角色以它的乐观态度和对朋友的忠诚而广受好评。\n\n除了作为一部著名的动画系列外，《哆啦A梦》还衍生出了多部电影、电视剧集以及一系列的玩具产品和其他媒体形式的产品开发项目。“哆啦A梦”已经成为一个国际性的文化现象和品牌标识的一部分。'], ['还有呢？', '图中展示的是一个哆啦A梦的表情包图片：\n\n- 哆啦A梦：一只蓝白色的卡通猫咪形象；\n  - 来自未来的机器人猫；\n  - 拥有四次元口袋和各种神奇道具；\n  - 助助主人公野比大雄度过困境；\n  \n该表情包通过哆啦A梦的形象传达了积极向上、乐于助人的精神风貌和大雄与哆啦A梦之间的深厚友谊情感共鸣。']]
GLM-4V:当然可以为您提供更多关于哆啦A梦的信息：
1. **背景设定**：《哆啦A梦》（原名《ドラえもん》）是由藤本弘（笔名藤子·F·不二雄）创作的一部著名日本漫画作品，
首次连载于1970年至1996年间。这部漫画讲述了从22世纪来到21世纪的机器猫哆啦A梦如何帮助他的朋友——小学生野比大雄克服困难的故事。
2. **主要人物**：
   - **哆啦A梦**：主角，是一只善良的机器猫，经常携带众多神奇的道具来帮助朋友们解决问题。
   - **野比大雄**：哆啦A梦的朋友，因为常犯错误而被同学们嘲笑，但心地善良且努力上进。
3. **特色道具**：哆啦A梦有许多神奇的道具，如“任意门”、“时光机”、“翻译面具”、
“缩小灯”、“百宝袋”（四次元口袋）、“照相机电话”等这些道具反映了作者对未来科技的想象和创新思维，同时也给读者带来了无尽的想象力空间。
4. **影响深远**：《哆啦A梦》不仅在日本国内取得了巨大成功，还在亚洲其他国家及世界各地赢得了广泛的粉丝群体。此外，根据原作改编的同名动画片也在全世界范围内播放并受到欢迎，“哆啦A梦”这一形象深入人心，成为了家喻户晓的文化符号之一